In [7]:
import pytorch_lightning as L

In [8]:
from data_loading import load_data, get_loaders, get_probe_data, get_probe_loaders, make_dataset
from transformer_predictor import TransformerPredictor, train, test

In [9]:
dataset = load_data('brackets.csv')

In [10]:
train_loader, val_loader, test_loader = get_loaders(dataset, batch_size=64)

In [11]:
model = TransformerPredictor(
    input_dim=4,
    model_dim=128,
    num_classes=2,
    num_heads=4,
    num_layers=2,
    lr=1e-3,
    warmup=100,
    max_iters=1000,
)
trainer = L.Trainer(max_epochs=10)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/akshitsinha3/Library/CloudStorage/OneDrive-InternationalInstituteofInformationTechnology/3-2/RSAI/Mechanistic_Interpretability/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [12]:
model, trainer, train_outbeddings, val_outbeddings = train(model, trainer, train_loader, val_loader)


  | Name                | Type               | Params
-----------------------------------------------------------
0 | input_net           | Sequential         | 640   
1 | positional_encoding | PositionalEncoding | 0     
2 | transformer         | TransformerEncoder | 264 K 
3 | output_net          | Sequential         | 17.0 K
4 | accuracy            | BinaryAccuracy     | 0     
-----------------------------------------------------------
282 K     Trainable params
0         Non-trainable params
282 K     Total params
1.131     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/akshitsinha3/Library/CloudStorage/OneDrive-InternationalInstituteofInformationTechnology/3-2/RSAI/Mechanistic_Interpretability/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/akshitsinha3/Library/CloudStorage/OneDrive-InternationalInstituteofInformationTechnology/3-2/RSAI/Mechanistic_Interpretability/transformer_predictor.py:277: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  accuracy = self.accuracy(F.softmax(y_hat), y)
/Users/akshitsinha3/Library/CloudStorage/OneDrive-InternationalInstituteofInformationTechnology/3-2/RSAI/Mechanistic_Interpretability/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
test_outbeddings = test(model, trainer, test_loader)

In [ ]:
X_probe_train, y_probe_train = get_probe_data(train_outbeddings)
X_probe_val, y_probe_val = get_probe_data(val_outbeddings)
X_probe_test, y_probe_test = get_probe_data(test_outbeddings)

train_dataset = make_dataset(X_probe_train, y_probe_train)
val_dataset = make_dataset(X_probe_val, y_probe_val)
test_dataset = make_dataset(X_probe_test, y_probe_test)

train_probe_loader, val_probe_loader, test_probe_loader = get_probe_loaders(train_dataset, val_dataset, test_dataset, batch_size=64)

In [ ]:
from probe import Probe

In [ ]:
probe = Probe(model_name='lr')
probe.fit(train_probe_loader, val_probe_loader)
probe.probe(X_probe_train, y_probe_train, X_probe_test, y_probe_test)